In [2]:
import keras 
print keras.__version__

Couldn't import dot_parser, loading of dot files will not be possible.
1.0.6


Using Theano backend.


# Notes for Keras (1.0.6) Source Code

## why
- Notes taken when I am trying to understand the code
- Provides very high-level snapshot of the source code structure
- Serves as roadmap for future references

## what
- Covers core packages so far, namely, `backend`, `engine`, `layers`, `models`, `optimizers`, `objectives`
- Big picture about the packages, taking LEGO as an analogy
    - `backend` defines LEGO blocks of different shapes. You can build combined blocks by connecting them in different ways. It's the foundation of other packages.
    - `engine`, in its two subpackage `topology` and `training`, defines two most important classes - `Layer` and `Model` classes. These are the parents of all other layers and models living in Keras.
    - `layers` and `models` packages contain all kinds of templates, which are similiar to the LEGO "recipes". Of course you can define your own. Just make sure to stick with the API so it works with other parts of Keras.
    - finally, `optimizers` package with others e.g., `objectives`, `regularizers`, `metrics` make your LEGO building alive.
    - It is really fun to see how the code organization in Keras encourages **open to extensions and close to modifications** design.
    
## how
- There might be mistakes and typos, because it is just my understanding as a learnner. Please bear this in mind if you read it.
- I found it really helpful to my understanding when I look at the test code first.

## 0. conventions
- shapes and dtypes are the most important specs for layers and variables in Keras. Their general meanings are,
    - `batch_input_shape`: the most general shape information, its first element should always be the _batch_size_, followed by the dimensions of actual inputs. It is useful in some models (e.g., stateful RNN) where batch_size needs to be explicitly spelled
    - `input_shape`: specifies the shape of input/output, without a `batch_size` (or = None). Common data used in Keras include,
        - flat vectors, so `input_shape = (, vec_dim)`, e.g., for Dense
        - sequence of integers, so `input_shape = (, max_int)`, e.g., for Embedding
        - sequence of vectors (e.g., sentence), so `input_shape = (nwords, word_vec_dim)`, e.g., for LSTM
        - images (in theano format), so `input_shape = (ncolors, width, height)`,e.g., for Conv2D
        - sequence of images (videos)? ,,,
    - `input_dim` is just a shortcut variable to specify `input_shape` is a single number. If the `input_shape` is a tuple of 2, it is usually specified by `input_dim` and `input_length` (sequence length) 
- most of time you just need to specify input shape information in the first layer (input layer) of the model

## 1. `backend` package
- in keras/backend folder, including 
    - `__init__.py`: exposes common interface for backend variables and operations
    - `common.py`: utility functions common for both tf and th backends 
    - `tensorflow_backend.py`: tf implementation of the common interface
    - `theano_backend.py`: th implementation
- the common interface implemented in tf and th backend are mostly
    - setter/getter of singleton varaibles, e.g., `_LEARNING_PHASE`, `_SESSION`, most of which are private (hidden from interface to users)
    - factory methods of `tensors`. There are roughly two types of tensors in tf/th libraries, which are exposed in Keras
        - `variable tensor`: tensors that you might have a value for. The value will be the state of the tensor, e.g., weights in your model. **Those are usually used as states of your model.**
        - `placeholder tensor`: tensors that you only have information of shapes about. Examples are training batches as they will be changed at each iteration - you don't have to maintain their states, you just need their shapes when building the model. **Those are usually used as inputs to your model.**
    - operators: the way that you can combine to generate new tensors on existing ones, e.g., gradient, linear-algebra, shape-manipulation and even rnn.
- Keras `backend` package summarizes very well what to expect in lower-level deep learning libraries: _it is all about tensors and their computation dependencies._
- One attractive feature of Keras is to directly use the backend API (K api). Because the layers and models expose both an object-oriented and functional interface, it is very intuitive to shift from the two levels. One good example is Keras's [neural_style_transfer](https://github.com/fchollet/keras/blob/master/examples/neural_style_transfer.py) code

## 2. `engine.topology` package

- the package conceptually defines how the computation flows in Keras.
- if you have ever read about [how simple a layer is implemented in theano](http://deeplearning.net/tutorial/mlp.html#mlp), you might first find it surprising why so much code were put in Keras layers implementations.

### 2.1  `Keras Tensor`, `Node` and `Layer` 
- Four main kinds of objects in `topology` are `Keras Tensor` (as described in doc string of `Input()` method), `Node`, `Layer` and `Container`
- A good reference is Keras's [test code](https://github.com/fchollet/keras/blob/master/tests/keras/engine/test_topology.py)
- `Keras Tensor` are the variables that actually flow in Keras. 
    - As described in the code, A `Keras tensor` is a tensor object from the underlying backend (Theano or TensorFlow), which we augment with certain attributes that allow us to build a Keras model just by knowing the inputs and outputs of the model.
    - The main "extra" attributes are `._keras_shape` and `._keras_history`, which are used to track the computing history of these tensors (a graph of `Node`s and `Layer`s)
    - The biggest gain is, given a set of inputs and outputs Keras variables, Keras framework should be able to automatically infer the flow from inputs to outputs, and thus construct the model based on it.
- The two most common ways of constructing a `Keras Tensor` in Keras
    - Use `Input()` method to create a variable from nothing - those will be the initial batch of tensor variables in Keras model. Note the creation of any `Keras Tensor` will usually create additional `Node` structures, which will be mentioned later. When a tensor is created by `Input`, an `Input Layer` with a `Node` (input layer's inbounding node) will be automatically created for consistency.
    - Use a layer's `__call__()` method to create an output `Keras Tensor` from an existing one. This involvs creating a `layer` first to implement the mapping from inputs to outputs. When the `layer` is actually called to a `tensor`, a new `node` will be created to connect the inputs and outputs `tensors`. I will talk about this later.
- A `Node` is literally like a node in the computing graph - it helps track the dependency and flow of computings/tensors. It's also the "physical" host of a mapping from input tensors to output tensors.
    - As a general rule, a node will be created every time when there is a mapping from some input tensors to one/more output tensors - this usually happens when the output tensors are created via a layer's `__call__()`
    - This node caputres the shape, mask, upstream layers and downstream layers information about the mapping from related input tensors to outputs.
    - `Nodes` are glue that make `Keras Tensors` trackable.
    - Multiple nodes should be created for multiple input-output-mappings, even though they might use the same layers for computational logic. And that layer will have multiple inbounding and outbounding nodes.
    - On the other hand, even though a layer can have mulitple inbound layers, it can only have one outbound layer, which is the one used to map the input tensors to output tensors in the node.
- A `Layer` in Keras is a "lightweight" structure which mainly focuses on implementing computing logics from input tensors and output tensors.
    - As a result, a `Layer` doesn't "physically" tracks its inputs and outputs (_they are tracked in layers' inbounding nodes_), and they can be reused multiple times in a computing graph for separate input-output mappings.
    - When a layer is re-used, it may have multiple inbound nodes and outbound nodes, each for different mappings.
    - The input(s) and output(s) of a layer can be queried by looking at its inbound nodes. 
- Relations between `Node`, `Layer` and `Tensor`
    - `Node` vs `Tensor`
        - each node is a mapping from input to output tensors. Those are the node's member variables, with masking and shape information
        - each tensor can be tracked a tuple of three (parent_layer, node_id, tensor_id). 
        - The input tensor of a node is the output tensor of its parent node, which can be found in the current node's inbound layer's inbound node's output tensor.
        - The output tensor of a node is created by applying the node's outbound layer to its input tensors
    - `Node` vs `Layer`
        - nodes are connected first to layers, and then to other nodes
        - ***roughly, it can be pictured as: in a graph (DAG) of nodes, (inbound) layers are their edges, as they define how tensors flow from one node to another.*** 
        - ***A node's inbound layers are connected to upstream nodes, and its outbound layer (single one) defines the logic of how inputs should be mapped to outputs.***
    - `Tensor` vs `Layer`
        - it would be natural to think that tensors belong to layers like in other libraries, but they are actually connected by nodes in Keras
    - The graph of `tensor`, `layer` and `node` is illustrated as below,
    ![tensor-layer-node](tensor-layer-node.png)
- Tracking the computational objects in Keras is usually done by:
    - This is implemented in the `__init__()` and `build_map_of_graph()` method of the `Container` class, which is the superclass of `Model`. It takes a list of input keras tensors and output keras tensors, and make a model by tracing the computation flow from inputs to outputs based on `Node` and `Layer` structures

### 2.2 What does a layer need to function properly?

### 2.3 Write your own Keras layers
- [documentation](https://keras.io/layers/writing-your-own-keras-layers/) and [simple example](https://github.com/fchollet/keras/blob/master/examples/antirectifier.py)
- with help of code in Layer class, it boils down to implementing 4 functions
    - `__init__(self, output_dim, **kwargs)`
        - it is OK to ignore it if your layer doesn't have extra states to initialize, so `Layer.__init__()` will be automatically called.
        - if you implement it, make sure to call `super(MyLayer,self).__init__(**kwargs)` otherwise the inner mechanism won't work.
    - `build(self, input_shape)`
    - `call(self, x, mask = None)`
    - `get_output_shape_for(self, input_shape)`

In [36]:
x = keras.layers.Input(shape = (1, ))
y = keras.layers.Lambda(lambda x: x+5)(x)
z = keras.layers.Lambda(lambda y: y*2)(y)
model = keras.models.Model(input = [x], output = z)

In [37]:
import numpy as np
model.predict([np.array([10])])

array([[ 30.]], dtype=float32)